## **Install Packages**

In [ ]:
!pip install langchain
#framework that use LLM and tools to build agent
!pip install langchain_openai
#bridge between langchain and openai
!pip install faiss-cpu
#for semantic search in vector database
!pip install pypdf
#for pdf loading
!pip install openai
#to access openai model
!pip install rank_bm25
#to use Bm25 keyword based searching algorithm

## **Import Libraries**

In [ ]:
from langchain.document_loaders import PyPDFLoader
#Loads and extracts text content from PDF files into LangChain-compatible documents.
from langchain.text_splitter import RecursiveCharacterTextSplitter
#It tries to split by larger structures first (like paragraphs), and if that fails, it recursively falls back to smaller structures (like sentences, then words, then characters)
from langchain.vectorstores import FAISS
from langchain.embeddings import OllamaEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
#Builds a QA chain that retrieves context from a vector database and maintains conversation history.
from langchain.memory import ConversationBufferMemory
#Stores and manages dialogue history across multiple turns of conversation.


## **Mounting Google Drive for PDF loading**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## **Load Two PDF from Google Drive**

In [ ]:
loader1 = PyPDFLoader('/content/drive/MyDrive/Colab Notebooks/Machine_Learning.pdf',)
loader2 = PyPDFLoader('/content/drive/MyDrive/Colab Notebooks/Mental_Health.pdf')

In [ ]:
docs1 = loader1.load()
docs2 = loader2.load()

## **Split the document into chunks**

In [ ]:
# Split into chunks
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100)


In [ ]:
chunk1 = splitter.split_documents(docs1)
chunk2 = splitter.split_documents(docs2)

## **Analyzing Chunks**

In [ ]:
print("Length of the first chunk=",   len(chunk1))
print("Length of the second chunk=", len(chunk2))

In [ ]:
print(chunk1)

In [ ]:
for chunk in chunk1:
  print(chunk.page_content)


In [ ]:
for i , chunk in enumerate(chunk1):
  print(f"Chunk {i+1}:\n {chunk.page_content} \n")
  print("-"*100)

In [ ]:
print(chunk2)

In [ ]:
for chunk in chunk2:
  print(chunk.page_content)

In [ ]:
for i , chunk in enumerate(chunk2):
  print(f"Chunk {i+1}:\n {chunk.page_content}\n")
  print("="*100)

## **Combining two PDF to create one Chunk**

In [ ]:
chunk_document = chunk1 + chunk2

In [ ]:
print(chunk_document)

In [ ]:
print("Length of chunk document", len(chunk_document))

In [ ]:
for chunk in chunk_document:
  print(chunk.page_content)

In [ ]:
for i, chunk in enumerate(chunk_document):
  print(f"Chunk {i+1} : \n {chunk.page_content}\n")
  print("+"*100)

In [ ]:
for chunk in chunk_document:
  print(chunk.metadata)

## **Create Ollama Embedding model**

In [ ]:
#ollama installation
!sudo apt update
!sudo apt install -y pciutils
!curl -fsSL https://ollama.com/install.sh | sh

In [ ]:
import threading
import subprocess
import time

def run_ollama_serve():
  subprocess.Popen(["ollama","serve"])

thread=threading.Thread(target=run_ollama_serve)
thread.start()
time.sleep(5)

In [ ]:
!pip install ollama

In [ ]:
!ollama pull all-minilm

In [ ]:
embedding_model=OllamaEmbeddings(model="all-minilm")

## **Create a Semantic Vector Store to convert text into vector**

In [ ]:
#convert text into vectors
semantic_vector_store = FAISS.from_documents(chunk_document, embedding_model)

In [ ]:
#save it to local disk
semantic_vector_store.save_local("faiss_index_semantic_vectorstore")

In [ ]:
# Load FAISS and LLM
semantic_vectorstore = FAISS.load_local(
    "faiss_index_semantic_vectorstore",
    embedding_model,
    allow_dangerous_deserialization=True)

## **Creating Semantic Retriever**

In [ ]:
semantic_retriever = semantic_vectorstore.as_retriever(
    search_kwargs={
        "k": 5,
        "filter": {
            "source": {"$in": ["machine_learning", "mental_health"]}
            #metada filtering
        }
    }
)

## **Create Memory**

**ConversationBufferMemory** stores the full conversation history in memory and feeds it into the LLM as context to enable multi-turn conversations.<br>
<br>
**memory_key="chat_history"**<br>
➤ This is the key used to store and retrieve previous messages in the conversation dictionary passed to the chain.
<br>
<br>
**return_messages=True**<br>
➤ This returns the conversation as ChatMessage objects ({"role": "user", "content": ...}), not just as one big string — better for chat models like GPT.
<br>
<br>
**output_key="answer"**<br>
➤ This tells memory to look for the LLM’s answer under the key "answer" in the chain’s output so it can append it to the history.




In [ ]:
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
    output_key="answer")

## **Creating LLM**

In [ ]:
import os

token=input("Enter your Github Token : ")
os.environ["GITHUB_TOKEN"]=token
model_name="openai/gpt-4.1-nano"
endpoint= "https://models.github.ai/inference"

llm=ChatOpenAI(
    model=model_name,
    api_key=token,
    base_url=endpoint,
    temperature=0.7
)

## **Create lexical store for keyword retriever**

**BM25Retriever**:<br>
-->smart keyword search engine<br>
-->No need for word embeddings or AI model<br>
-->find most relevant chunks of text based using BM25 algorithm<br>
-->previously we do manual tokenizing,bm-scoring,sorting ,listing top k but this retriever includes all internally
<br>
<br>


In [ ]:
from langchain.retrievers import BM25Retriever

#Create keyword based  Retriever
keyword_retriever = BM25Retriever.from_documents(chunk_document)
keyword_retriever.k = 5

## **Create Hybrid Retriever for hybrid search**

**EnsembeleRetriever:**<br>
----combines two or more retriver<br>
----weights control the importance of each retriever<br>
----normalize scores and return top matchings

In [ ]:
from langchain.retrievers import EnsembleRetriever

hybrid_retriever = EnsembleRetriever(
    retrievers=[keyword_retriever, semantic_retriever],
    weights=[0.5, 0.5]
)

## **Create a Chain that connects all**

In [ ]:
final_answer_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    #generate answers
    retriever=hybrid_retriever,
    #find relevant document
    memory=memory,
    #keeps track of previous question
    return_source_documents=True,
    #return chunks that were used to answer the question
    output_key="answer"
)


# **Testing**

In [ ]:
questions_document=[
    "What is Machine Learning?",
    "What are the types of it?",
    "Tell me about its algorithms?",
    "Which algorithm is suitable for supervised Learning?",
    "Tell me which tools and libraries are used for this?",
    "What is its real world application",
    "What is Mental Health?",
    "Tell me its importance",
    "What could be the common disorders beacause of this?",
    "What are the causes of Mental Health",
    "Does it curable?",
    "What are the treatments need to cure this?",
    "Are you free from Mental Disorder?"

]

In [ ]:
for question in questions_document:
    response = final_answer_chain.invoke({"question": question})
    print(f"❓: {question}\n🗨️: {response['answer']}\n")

In [ ]:
!pip install gradio --quiet

In [ ]:
import gradio as gr

In [ ]:
#Gradio chatbot UI function
chat_history = []

def chatbot_response(user_input):
    global chat_history
    if user_input.strip() == "":
        return chat_history, ""
    result = qa_chain.invoke({"question": user_input})
    answer = result['answer']
    chat_history.append(("You", user_input))
    chat_history.append(("Bot", answer))
    # Format history for Gradio chatbot [(user_msg, bot_msg), ...]
    formatted_history = [(chat_history[i][1], chat_history[i+1][1]) for i in range(0, len(chat_history), 2)]
    return formatted_history, ""

In [ ]:
#Build Gradio interface
with gr.Blocks() as demo:
    gr.Markdown("## PDF-based Context-Aware Chatbot")
    with gr.Row():
        with gr.Column(scale=1, min_width=200):
            chatbot = gr.Chatbot(elem_id="chatbot", height=250)
            user_input = gr.Textbox(placeholder="Ask a question about the PDFs...", show_label=False)
            clear = gr.Button("Clear Chat")

            user_input.submit(chatbot_response, inputs=user_input, outputs=[chatbot, user_input])
            clear.click(lambda: ([], ""), None, [chatbot, user_input])


demo.launch()